### Set up LLM

In [1]:
#Using AzureOpenAI as LLM. Other LLM options are available in LlamaIndex
#from llama_index.llms.azure_openai import AzureOpenAI

# Using OpenAI as LLM with LlamaIndex
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from dotenv import load_dotenv
load_dotenv()
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

#API key for OpenAi or AzureOpenAI. Replace with your own key
api_key = os.environ.get("OPENAI_API_KEY")

#Azure Endpoint URL. Replace with your own URL
#azure_endpoint = "https://agentic-ai-course-kumaran.openai.azure.com/"

#Note : Function calling support only available in GPT-4+ (maybe also in GPT-3.5 Turbo)
# Create the LLM object for OpenAI
llm = OpenAI(
    model="gpt-4o",  # or "gpt-4-turbo", "gpt-3.5-turbo-16k", etc.
    api_key=api_key
)

#Create the LLM object for Azure OpenAI
#llm=AzureOpenAI(
#    model="gpt-4o",
#    deployment_name="gpt-4o",
#    api_key=api_key,
#    azure_endpoint=azure_endpoint,
#    api_version="2024-08-01-preview",
#)

# Optionally, set as default for LlamaIndex
#Settings.llm = llm


### 03.02. Create introspection agent

In [5]:
from llama_index.agent.introspective import SelfReflectionAgentWorker
from llama_index.agent.introspective import IntrospectiveAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole

#Setup the reflection agent worker
self_reflection_agent_worker = SelfReflectionAgentWorker.from_defaults(
    llm=llm,
    verbose=True,
)

#Setup the introspective agent worker
introspective_agent_worker = IntrospectiveAgentWorker.from_defaults(
    reflective_agent_worker = self_reflection_agent_worker,
    main_agent_worker = None,
    verbose=True,
)

#Create a system prompt defining the function of the agent
system_prompt="""
You are a Product specification summarizer who can summarize a product specification.
For the input provided, create a summary with less than 100 words.

Ensure that the summary focuses on performance specifications
and safety features, and that it doesn't copy-paste from a product specification, - rephrase.

If you will use any units of measurement, use the metric system and in square brackets reference the original value and its unit.
"""
chat_history = [
    ChatMessage(
        content=system_prompt,
        role=MessageRole.SYSTEM
    )
]

#Create the agent
introspective_agent= introspective_agent_worker.as_agent(
    chat_history=chat_history,
    verbose=True
)

### 03.03. Execute the introspection Agent

In [6]:
from llama_index.readers.file import PyMuPDFReader

#Input file for summarization
loader=PyMuPDFReader()
docs=loader.load(file_path="./datasets/EcoSprint_Specification_Document.pdf")


#Pick the first page of the doc as content
source_content=docs[0].text
print(source_content)




EcoSprint Specification Document
1. Overview
●
The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and
performance. With its sleek design and state-of-the-art technology, the EcoSprint
appeals to environmentally conscious drivers who don't want to compromise on style or
driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a
perfect blend of comfort, sustainability, and innovation.
2. Design Specifications
●
Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring
smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean
Blue, and Pearl White, it's a head-turner on the road.
●
Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a
spacious cabin with seating for five, premium upholstery, and customizable ambient
lighting.
3. Performance Specifications
●
Engine and Motor: Powered by a high-efficiency electric motor, the EcoSprint delivers
200 ho

In [7]:
response=await introspective_agent.achat(source_content)

print("\n", response.response)

> Running step 90ee242e-0214-4787-bf51-9296f4ef53d0. Step input: EcoSprint Specification Document
1. Overview
●
The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and
performance. With its sleek design and state-of-the-art technology, the EcoSprint
appeals to environmentally conscious drivers who don't want to compromise on style or
driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a
perfect blend of comfort, sustainability, and innovation.
2. Design Specifications
●
Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring
smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean
Blue, and Pearl White, it's a head-turner on the road.
●
Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a
spacious cabin with seating for five, premium upholstery, and customizable ambient
lighting.
3. Performance Specifications
●
Engine and Motor: Powered b